In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('riceClassification.csv')

In [4]:
data.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
#memeriksa missing value

data.isna().sum()

id                 0
Area               0
MajorAxisLength    0
MinorAxisLength    0
Eccentricity       0
ConvexArea         0
EquivDiameter      0
Extent             0
Perimeter          0
Roundness          0
AspectRation       0
Class              0
dtype: int64

In [6]:
data.describe()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
count,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000,18185.000000
mean,9093.000000,7036.492989,151.680754,59.807851,0.915406,7225.817872,94.132952,0.616653,351.606949,0.707998,2.599081,0.549079
std,5249.701658,1467.197150,12.376402,10.061653,0.030575,1502.006571,9.906250,0.104389,29.500620,0.067310,0.434836,0.497599
min,1.000000,2522.000000,74.133114,34.409894,0.676647,2579.000000,56.666658,0.383239,197.015000,0.174590,1.358128,0.000000
25%,4547.000000,5962.000000,145.675910,51.393151,0.891617,6125.000000,87.126656,0.538530,333.990000,0.650962,2.208527,0.000000
50%,9093.000000,6660.000000,153.883750,55.724288,0.923259,6843.000000,92.085696,0.601194,353.088000,0.701941,2.602966,1.000000
75%,13639.000000,8423.000000,160.056214,70.156593,0.941372,8645.000000,103.559146,0.695664,373.003000,0.769280,2.964101,1.000000
max,18185.000000,10210.000000,183.211434,82.550762,0.966774,11008.000000,114.016559,0.886573,508.511000,0.904748,3.911845,1.000000


In [7]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size = 0.33, random_state = 42)

In [8]:
#ukuran data latih

train.shape

(12183, 12)

In [9]:
#ukuran data uji

test.shape

(6002, 12)

In [10]:
#pemisahan atribut id dari dataset

train_id = train['id']
test_id = test['id']

train.drop(['id'], axis = 1, inplace = True)
test.drop(['id'], axis = 1, inplace = True)

In [11]:
def normalize(dat):
    l = []
    for i in dat:
        l.append((i - np.mean(dat)) / np.std(dat))
    return np.array(l)

In [12]:
train['Area'] = normalize(train['Area'])
train['MajorAxisLength'] = normalize(train['MajorAxisLength'])
train['MinorAxisLength'] = normalize(train['MinorAxisLength'])
train['ConvexArea'] = normalize(train['ConvexArea'])
train['EquivDiameter'] = normalize(train['EquivDiameter'])
train['Perimeter'] = normalize(train['Perimeter'])
train['AspectRation'] = normalize(train['AspectRation'])

test['Area'] = normalize(test['Area'])
test['MajorAxisLength'] = normalize(test['MajorAxisLength'])
test['MinorAxisLength'] = normalize(test['MinorAxisLength'])
test['ConvexArea'] = normalize(test['ConvexArea'])
test['EquivDiameter'] = normalize(test['EquivDiameter'])
test['Perimeter'] = normalize(test['Perimeter'])
test['AspectRation'] = normalize(test['AspectRation'])

In [13]:
train_class = train['Class']
test_class = test['Class']

In [24]:
import math
from random import randrange
from random import seed
 
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return math.sqrt(distance)
 
# Locate the best matching unit
def get_best_matching_unit(codebooks, test_row):
    distances = list()
    for codebook in codebooks:
        dist = euclidean_distance(codebook, test_row)
        distances.append((codebook, dist))
    distances.sort(key=lambda tup: tup[1])
    return distances[0][0]
 
# Create a random codebook vector
def random_codebook(train):
    n_records = len(train)
    n_features = len(train[0])
    codebook = [train[randrange(n_records)][i] for i in range(n_features)]
    return codebook
 
# Train a set of codebook vectors
def train_codebooks(train, n_codebooks, lrate, epochs):
    codebooks = [random_codebook(train) for i in range(n_codebooks)]
    for epoch in range(epochs):
        rate = lrate * (1.0-(epoch/float(epochs)))
        sum_error = 0.0
        for row in train:
            bmu = get_best_matching_unit(codebooks, row)
            for i in range(len(row)-1):
                error = row[i] - bmu[i]
                sum_error += error**2
                if bmu[-1] == row[-1]:
                    bmu[i] += rate * error
                else:
                    bmu[i] -= rate * error
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, rate, sum_error))
    return codebooks
 
# Test the training function
seed(1)
learn_rate = 0.3
n_epochs = 10
n_codebooks = 2
codebooks = train_codebooks(train.values.tolist(), n_codebooks, learn_rate, n_epochs)
print('Codebooks: %s' % codebooks)

>epoch=0, lrate=0.300, error=44587.655
>epoch=1, lrate=0.270, error=43439.932
>epoch=2, lrate=0.240, error=42576.760
>epoch=3, lrate=0.210, error=41749.277
>epoch=4, lrate=0.180, error=40980.047
>epoch=5, lrate=0.150, error=40225.948
>epoch=6, lrate=0.120, error=39515.669
>epoch=7, lrate=0.090, error=38827.022
>epoch=8, lrate=0.060, error=38176.509
>epoch=9, lrate=0.030, error=37523.415
Codebooks: [[-0.7363752095778924, -0.021150024848569482, -0.8859579640822896, 0.9408257670052569, -0.7367030275620265, -0.7286975571449371, 0.5789624685916585, -0.44037791431015055, 0.6525096742489385, 0.8680936990242935, 1.0], [0.9701920805891753, 0.14741150938886408, 1.114751333020292, 0.8851345902322484, 0.9616422603839103, 0.96266779271149, 0.6501349662159763, 0.596466013004075, 0.7787304144512662, -1.0057242385525225, 0.0]]


In [25]:
#apply codebook to test dataset
res = []

for i in test.index:
    total_0 = 0
    total_1 = 0
    for j in range(0, len(test.loc[i] - 1)):
        total_0 += codebooks[0][j] * test.loc[i][j]
        total_1 += codebooks[1][j] * test.loc[i][j]
    if total_0 < total_1:
        res.append(0)
    else:
        res.append(1)

In [26]:
#count the accuracy score and plot confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy = accuracy_score(res, test['Class'])
print(accuracy)
confusion_matrix(res, test['Class'])

0.9726757747417527


array([[2561,   14],
       [ 150, 3277]], dtype=int64)